In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
OPEN_AI_MODEL = os.getenv("OPEN_AI_MODEL")

# instantiate the openAI client
client = OpenAI(api_key=OPEN_AI_API_KEY)

We first need to upload the file powering our RAG in a [Vector Store](https://platform.openai.com/docs/api-reference/vector-stores).

In [2]:
def upload_file_to_vector_store(path):
    # upload a file with an "assistants" purpose
    file = client.files.create(file=open(path, "rb"), purpose="assistants")
    # create a vector store
    vector_store = client.vector_stores.create(
        name="Some Fake Football Articles"
    )
    # add the file to the vector store just created
    client.vector_stores.files.create(
        vector_store_id=vector_store.id,
        file_id=file.id
    )

    # return the vector store (we'll need the ID)
    return vector_store

vector_store = upload_file_to_vector_store("data/fake_football_articles.json")

We can start interacting with our RAG, using the [Responses API](https://platform.openai.com/docs/api-reference/responses).

In [3]:
def generate_response(prompt):
    response = client.responses.create(
        model=OPEN_AI_MODEL,
        input=prompt,
        instructions="You need to perform a file search of the provided document, and summarise all the articles you can find about the player provided by the user, if there's any (otherwise, simply say 'No articles on the provided player'). Focus on strengths and weaknesses, as well as the evolution of the judgement over time in case of multiple articles.",
        tools=[
            {
                "type": "file_search",
                "vector_store_ids": [vector_store.id], # vector store containing our file
            },
        ]
    )
    # create the string message
    new_message = ' '.join([r.content[0].text for r in response.output if r.type == 'message'])

    return new_message

In [5]:
response = generate_response("Is Thierry Doumbia a good player?")

In [6]:
print(response)

### Overview of Thierry Doumbia's Performance and Evaluation

Thierry Doumbia has recently been the subject of several articles, which highlight both his strengths and weaknesses as a player:

1. **Strengths**:
   - **Acceleration and Speed**: Doumbia is noted for his exceptional acceleration while playing for Bamako Rangers, making him a significant challenge for defenders on the field.
   
2. **Weaknesses**:
   - **Technical Limitations**: While his athleticism is praised, he lacks the technical finesse required to match it, leading to frequent turnovers in tight situations.

3. **Evolution of Judgement**:
   - **Potential Growth**: There are links suggesting interest from clubs in the Zandora League, where his raw pace could be an asset. This indicates some clubs see potential in his abilities, despite the noted limitations.

### Conclusion
Overall, Thierry Doumbia is acknowledged for his speed and acceleration but is critiqued for lacking technical skills, which affects his perform